# Programming for Data Science and Artificial Intelligence

## 0. Linear Regression Extension

You have learned all the detail of Linear Regression. Its core idea is presented in the lecture.

To enhance the `Gradient` function, there are two extensions that improve the optimization process in terms of quality of life (your life).

1. Early Stopping
2. Momentum

In [1]:
#  Code from the class
import numpy as np
from sklearn.model_selection import KFold


def mean_squared_error(ytrue, ypred):
    return ((ypred - ytrue) ** 2).sum() / ytrue.shape[0]

class LinearRegression_v1(object):
    
    kfold = KFold(n_splits=5)
            
    def __init__(self, alpha=0.001, num_epochs=5, batch_size=50, method='batch',
                 cv=kfold):
        self.alpha      = alpha
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.method     = method
        self.cv         = cv
    
    def fit(self, X_train, y_train):
        
        #using training......
        
        #please change it to cross-validation.....
        
        #create a list of kfold scores
        self.kfold = list()

        #Kfold.split in the sklearn.....
        #5 splits
        for fold, (train_idx, val_idx) in enumerate(self.cv.split(X_train)):
            
            X_cross_train = X_train[train_idx]
            y_cross_train = y_train[train_idx]
            X_cross_val   = X_train[val_idx]
            y_cross_val   = y_train[val_idx]
            
            #create self.theta here
            self.theta = np.zeros(X_cross_train.shape[1])
            
            #define X_cross_train as only a subset of the data
            #how big is this subset?  => mini-batch size ==> 50
            
            #one epoch will exhaust the WHOLE training set
            for epoch in range(self.num_epochs):
            
                #with replacement or no replacement
                #with replacement means just randomize
                #with no replacement means 0:50, 51:100, 101:150, ......300:323
                #shuffle your index
                #===> please shuffle your index
                perm = np.random.permutation(X_cross_train.shape[0])
                        
                X_cross_train = X_cross_train[perm]
                y_cross_train = y_cross_train[perm]
                
                if   self.method == 'sto':
                    for batch_idx in range(X_cross_train.shape[0]):
                        X_method_train = X_cross_train[batch_idx].reshape(1, -1) #(11,) ==> (1, 11) ==> (m, n)
                        y_method_train = y_cross_train[batch_idx]                    
                        self._train(X_method_train, y_method_train)
                elif self.method == 'mini':
                    for batch_idx in range(0, X_cross_train.shape[0], self.batch_size):
                        #batch_idx = 0, 50, 100, 150
                        X_method_train = X_cross_train[batch_idx:batch_idx+self.batch_size, :]
                        y_method_train = y_cross_train[batch_idx:batch_idx+self.batch_size]
                        self._train(X_method_train, y_method_train)
                else:
                    X_method_train = X_cross_train
                    y_method_train = y_cross_train
                    self._train(X_method_train, y_method_train)
                    
            yhat_val = self.predict(X_cross_val)
            self.kfold.append(mean_squared_error(y_cross_val, yhat_val))
            print(f"Fold {fold}: {mean_squared_error(y_cross_val, yhat_val)}")
                    
    def _train(self, X, y):
        yhat = self.predict(X)
        grad = X.T @(yhat - y)
        self.theta = self.theta - self.alpha * grad
    
    def predict(self, X):
        return X @ self.theta  #===>(m, n) @ (n, )
    
    def _coef(self):
        return self.theta[1:]  #remind that theta is (w0, w1, w2, w3, w4.....wn)
                               #w0 is the bias or the intercep
                               #w1....wn are the weights / coefficients / theta
        
    def _bias(self):
        return self.theta[0]
    
    

## 1. Early Stopping

`num_epochs` is the parameter that specify how many iteration your model will perform the optimization.

The question to ask is "How many `num_epochs` is enough?".

The answer is when your model is optimized.

Optimized? What is optimized?

Ah.. young padawan, remember this figure?

<img width="400" src = "figures/gradient.png">

We say the model is trained/optimized/learned when *it* reachs the minima, preferably, global minima.

Our goal of optimization in this Gradient context is to minimize the loss/error.

$$ \min_{\theta} || \hat{y} - y ||^2_2 $$

$\theta$ is vector of weights to optimize.

$\hat{y}$ is actually a result of $ \mathbf{X}\theta $.

$y$ is a vector of ground-truth/label.

$|| . ||^2_2$ is the loss/error function. 

Now, you remember your objective. It is to minimize the $|| . ||^2_2$ by adjusting the $ \theta $

Then, the model is done training when you finally get the $ \theta $ that yeild the best result (lowest error / minima).

### How do you know you are there at the minima?

Well, one way is to tracking your loss over the training session. 

![loss](https://miro.medium.com/max/411/1*UHmMMH3OhrBvgw18jKiy6Q.png)

You see, when you plot the loss/error (in the figure is cost) over the epochs, it should be desending and finally plateau.

When it is plateau, in other words, adjusting $\theta$ no longer improve the prediction. You are at the minima. (Gradient is 0 at minima).

According to the plot of lost/error, when do we reach the minima? 3000?, 2000?, 1000?

In this case, do you need to set `num_epochs` to 3000?

In the next dataset you will work on, what `num_epochs` you need to set?

Ah.. You will never know until you train the model with that new dataset, at least, one time.

Can we do better?

Yes, we can. Introducing!!!!!!!!! `Early Stopping`!!!!!!!!!

### Early Stopping

The idea of `early stoping` is, as the name suggest, break the training before it reachs the `num_epochs`.

The key is **no longer improve**. **Improve!!**. What is improve?

Improve is the loss of iteration/epoch *i* is lower than the *i-1*.

How much is the $\text{loss}_i - \text{loss}_{i-1}$ so that we say it is improving? This is up to you. 

Now code!!!


In [ ]:
### pesudo code for people who has no idea

# for _ in range(epochs)
#   train(x_train, y_train)
#   loss = error(predict(x_train) - y_train)
#   if prev_loss - loss > theshold
#      ok, there is an improment
#      prev_loss = loss
#   else
#      done training. break

In [117]:
import numpy as np
from sklearn.model_selection import KFold


def mean_squared_error(ytrue, ypred):
    return ((ypred - ytrue) ** 2).sum() / ytrue.shape[0]

class LinearRegression_v2(object):
    
    kfold = KFold(n_splits=5)
            
    def __init__(self, alpha=0.001, num_epochs=5, batch_size=50, method='batch',
                 cv=kfold, theshold=0.000001):
        self.alpha      = alpha
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.method     = method
        self.cv         = cv
        self.theshold   = theshold
    
    def fit(self, X_train, y_train):
        
        #using training......
        
        #please change it to cross-validation.....
        
        #create a list of kfold scores
        self.kfold = list()

        #Kfold.split in the sklearn.....
        #5 splits
        for fold, (train_idx, val_idx) in enumerate(self.cv.split(X_train)):
            
            X_cross_train = X_train[train_idx]
            y_cross_train = y_train[train_idx]
            X_cross_val   = X_train[val_idx]
            y_cross_val   = y_train[val_idx]
            
            #create self.theta here
            self.theta = np.zeros(X_cross_train.shape[1])
            
            #define X_cross_train as only a subset of the data
            #how big is this subset?  => mini-batch size ==> 50

            #Set previous loss to 0
            prev_loss = 0
            
            #one epoch will exhaust the WHOLE training set
            for epoch in range(self.num_epochs):
            
                #with replacement or no replacement
                #with replacement means just randomize
                #with no replacement means 0:50, 51:100, 101:150, ......300:323
                #shuffle your index
                #===> please shuffle your index
                perm = np.random.permutation(X_cross_train.shape[0])
                        
                X_cross_train = X_cross_train[perm]
                y_cross_train = y_cross_train[perm]
                
                if   self.method == 'sto':
                    for batch_idx in range(X_cross_train.shape[0]):
                        X_method_train = X_cross_train[batch_idx].reshape(1, -1) #(11,) ==> (1, 11) ==> (m, n)
                        y_method_train = y_cross_train[batch_idx]                    
                        self._train(X_method_train, y_method_train)
                elif self.method == 'mini':
                    for batch_idx in range(0, X_cross_train.shape[0], self.batch_size):
                        #batch_idx = 0, 50, 100, 150
                        X_method_train = X_cross_train[batch_idx:batch_idx+self.batch_size, :]
                        y_method_train = y_cross_train[batch_idx:batch_idx+self.batch_size]
                        self._train(X_method_train, y_method_train)
                else:
                    X_method_train = X_cross_train
                    y_method_train = y_cross_train
                    self._train(X_method_train, y_method_train)
                    
                #Early Stopping
                yhat_cross_train = self.predict(X_cross_train)
                loss = mean_squared_error(y_cross_train, yhat_cross_train)
                d_loss = abs(prev_loss - loss)
                if d_loss > self.theshold:
                    #ok, there is an improment
                    prev_loss = loss
                else:
                    #done training. break
                    break
                
            print(f"Loss threshold achieved at num epoch: {epoch}")

            yhat_val = self.predict(X_cross_val)
            self.kfold.append(mean_squared_error(y_cross_val, yhat_val))
            print(f"Fold {fold}: {mean_squared_error(y_cross_val, yhat_val)}")
                    
    def _train(self, X, y):
        yhat = self.predict(X)
        grad = X.T @(yhat - y)
        self.theta = self.theta - self.alpha * grad
    
    def predict(self, X):
        return X @ self.theta  #===>(m, n) @ (n, )
    
    def _coef(self):
        return self.theta[1:]  #remind that theta is (w0, w1, w2, w3, w4.....wn)
                               #w0 is the bias or the intercep
                               #w1....wn are the weights / coefficients / theta
        
    def _bias(self):
        return self.theta[0]

In [118]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()

X = diabetes.data
y = diabetes.target

#print the shape of X and y
X.shape, y.shape

((442, 10), (442,))

In [119]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=999)

# Assert X_train and y_train has same ammount of samples
assert X_train.shape[0] == y_train.shape[0]

# Assert X_test and y_test has same ammount of samples
assert X_test.shape[0] == y_test.shape[0]

In [120]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# Satndadize the trainning set
X_train = sc.fit_transform(X_train)

# Standardize the test set
X_test = sc.transform(X_test)

In [121]:
lr = LinearRegression_v2(method='batch',num_epochs=1000,theshold=0.01)

In [122]:
lr.fit(X_train, y_train)

Loss threshold achieved at num epoch: 846
Fold 0: 33323.04414751473
Loss threshold achieved at num epoch: 268
Fold 1: 28205.42602894653
Loss threshold achieved at num epoch: 763
Fold 2: 28455.087975256658
Loss threshold achieved at num epoch: 740
Fold 3: 29704.604613021125
Loss threshold achieved at num epoch: 693
Fold 4: 29511.55141799242


### Discussion: 
- Even though `num_epochs` is set to 1000, in each fold the requierd loss threshold of 0.01 is achieved in less than 1000 iterations (846, 268, 736, 740, 693 iterations in the above example) after implementing 'Early Stopping'

In [123]:
yhat = lr.predict(X_test)

In [124]:
mean_squared_error(y_test, yhat)

23594.633554793396

In [125]:
lr._coef()

array([ -7.8141485 ,  29.39367493,  16.61730099, -38.04383398,
        40.3691605 ,   1.16936199, -13.25776384,  37.06672119,
         3.39059881])

In [126]:
lr._bias()

-7.2467994208371955

## 2. Momentum

`alpha` or `learning_rate` is the parameter how big of the step you will take on each Gradient optimization step.

![overshoot](https://miro.medium.com/max/700/1*hGhRddOUV8h0pdQek8T35A.png)

How much is `alpha` you should set?

Not big such that it causes the *over shooting*.

And certainly, not too small such that it causes you extra epochs to reach the minima.

You want **just right** `alpha`.

How do you know what is the right `alpha`?

You guess, right?

This time, 0.1. Observe the loss. If it is overshooting, stop and reduce the `alpha`. If the loss reduce slightly, maybe stop and increase the `alpha`.

Finally, if you spend time adjusting the `alpha`, you will eventually find the **just right** `alpha`.

Can we do better? Can we reduce the number of epochs without searching for the **just right** `alpha`.

Yes, we can!!! Introducing!!!!!!!! MOMENTUM~~!!!!!!!!!

### Momentum

Here is the updating equation.

$$ \theta^{i+1} = \theta^{i} - \alpha \times \text{gradient}^i $$

$i$ is the iteration/epoch number.

$\theta^{i+1}$ is the updated weights and will be used for the next iteration.

$\theta^{i}$ is the current weights of iteration $i$.

$\alpha$ is the learning rate.

$\text{gradient}^i$ is the step suggesting by the deviation of loss function from iteration $i$.

$\alpha \times \text{gradient}^i$ can be called *step*. When consider $i$, we call the $step^i$ how big of the step you take in iteration $i$.

We know, in the early iteration, the step will be very big. Given the `alpha` is not too big, the step will decrase over time.

Let's say our step size was big in the previous iteration (i-1), maybe, instead of taking $step^i$, we can take bigger step than $step^i$.

Yes, and here is the equation.

$$ \theta^{i+1} = \theta^{i} - step^i + \text{momentum} \times step^{i-1} $$

Then, *momentum* here is how much do you want to include the information of previous step. The range of *momentum* is $[0.0 - 1.0]$. When *momentum* is 0 then you do not use previous step (basically, normal optimization). When it is 1 then you fully use the information from previous step. 

Now, code !!!!

In [127]:
### pesudo code for people who has no idea

# step = alpha * grad
# theta = theta - step + momentum * prev_step
# prev_step = step

In [162]:
import numpy as np
from sklearn.model_selection import KFold


def mean_squared_error(ytrue, ypred):
    return ((ypred - ytrue) ** 2).sum() / ytrue.shape[0]

class LinearRegression_v3(object):
    
    kfold = KFold(n_splits=5)
            
    def __init__(self, alpha=0.001, num_epochs=5, batch_size=50, method='batch',
                 cv=kfold, theshold=0.000001, momentum=0.01):
        self.alpha      = alpha
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.method     = method
        self.cv         = cv
        self.theshold   = theshold
        self.momentum   = momentum
        self.prev_step = 0
    
    def fit(self, X_train, y_train):
        
        #using training......
        
        #please change it to cross-validation.....
        
        #create a list of kfold scores
        self.kfold = list()

        #Kfold.split in the sklearn.....
        #5 splits
        for fold, (train_idx, val_idx) in enumerate(self.cv.split(X_train)):
            
            X_cross_train = X_train[train_idx]
            y_cross_train = y_train[train_idx]
            X_cross_val   = X_train[val_idx]
            y_cross_val   = y_train[val_idx]
            
            #create self.theta here
            self.theta = np.zeros(X_cross_train.shape[1])
            
            #define X_cross_train as only a subset of the data
            #how big is this subset?  => mini-batch size ==> 50

            #Set previous loss to 0
            prev_loss = 0
            
            #one epoch will exhaust the WHOLE training set
            for epoch in range(self.num_epochs):
            
                #with replacement or no replacement
                #with replacement means just randomize
                #with no replacement means 0:50, 51:100, 101:150, ......300:323
                #shuffle your index
                #===> please shuffle your index
                perm = np.random.permutation(X_cross_train.shape[0])
                        
                X_cross_train = X_cross_train[perm]
                y_cross_train = y_cross_train[perm]
                
                if   self.method == 'sto':
                    for batch_idx in range(X_cross_train.shape[0]):
                        X_method_train = X_cross_train[batch_idx].reshape(1, -1) #(11,) ==> (1, 11) ==> (m, n)
                        y_method_train = y_cross_train[batch_idx]                    
                        self._train(X_method_train, y_method_train)
                elif self.method == 'mini':
                    for batch_idx in range(0, X_cross_train.shape[0], self.batch_size):
                        #batch_idx = 0, 50, 100, 150
                        X_method_train = X_cross_train[batch_idx:batch_idx+self.batch_size, :]
                        y_method_train = y_cross_train[batch_idx:batch_idx+self.batch_size]
                        self._train(X_method_train, y_method_train)
                else:
                    X_method_train = X_cross_train
                    y_method_train = y_cross_train
                    self._train(X_method_train, y_method_train)
                    
                #Early Stopping
                yhat_cross_train = self.predict(X_cross_train)
                loss = mean_squared_error(y_cross_train, yhat_cross_train)
                d_loss = abs(prev_loss - loss)
                if d_loss > self.theshold:
                    #ok, there is an improment
                    prev_loss = loss
                else:
                    #done training. break
                    break
                
            print(f"Loss threshold achieved at num epoch: {epoch}")

            yhat_val = self.predict(X_cross_val)
            self.kfold.append(mean_squared_error(y_cross_val, yhat_val))
            print(f"Fold {fold}: {mean_squared_error(y_cross_val, yhat_val)}")
                    
    def _train(self, X, y):
        yhat = self.predict(X)
        grad = X.T @(yhat - y)
        step = self.alpha * grad
        self.theta = self.theta - (step + self.momentum * self.prev_step)
        self.prev_step = step
    
    def predict(self, X):
        return X @ self.theta  #===>(m, n) @ (n, )
    
    def _coef(self):
        return self.theta[1:]  #remind that theta is (w0, w1, w2, w3, w4.....wn)
                               #w0 is the bias or the intercep
                               #w1....wn are the weights / coefficients / theta
        
    def _bias(self):
        return self.theta[0]

In [169]:
lr = LinearRegression_v3(method='batch', num_epochs=1000, theshold=0.01, momentum=0.02)

In [170]:
lr.fit(X_train, y_train)

Loss threshold achieved at num epoch: 833
Fold 0: 33323.47826450363
Loss threshold achieved at num epoch: 266
Fold 1: 28205.068864447578
Loss threshold achieved at num epoch: 753
Fold 2: 28455.316725759985
Loss threshold achieved at num epoch: 729
Fold 3: 29704.682136980307
Loss threshold achieved at num epoch: 684
Fold 4: 29511.553635877586


### Discussion: 
- After introducing 'Momentum' of 0.02, in each fold the number of iteratins have reduced further from (846, 268, 736, 740, 693) to (833, 266, 753, 729, 684) in the above example.

In [165]:
yhat = lr.predict(X_test)

In [166]:
mean_squared_error(y_test, yhat)

23594.65594546635

In [167]:
lr._coef()

array([ -7.81438346,  29.39343876,  16.61740747, -38.06128459,
        40.38264417,   1.17721048, -13.25536685,  37.07334764,
         3.39048634])

In [168]:
lr._bias()

-7.24685663379659